<a href="https://colab.research.google.com/github/AliAbdallah21/LangGraph-QA-Workflow/blob/main/LangGraph_QA_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Installations
%pip install -q langgraph langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 13.9 MB/s eta 0:00:00


In [2]:
# @title Imports
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from google.colab import userdata

In [10]:
# @title API Disclaimer
api_key =userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(model = 'gpt-3.5-turbo', openai_api_key = api_key)

### **Building a QA Workflow Specific to the Guided Project**

<div style="text-align: center;">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BgP-ruk_KS5H8D7iISsJ6A/Screenshot%202024-12-20%20at%204-20-07%E2%80%AFPM.png" alt="Screenshot" width="150">
</div>


Now, we are designing a **Question-Answering (QA) workflow** specifically tailored for a guided project. This workflow leverages LangGraph to create modular, state-driven transitions and ensures that questions related to the guided project are prioritized and handled effectively.

The workflow evaluates whether a user’s query is relevant to the guided project (for example, what is this guided project about? or what is LangGraph?). For relevant questions, it uses predefined context to generate an informed response. If the query is unrelated to the guided project, the workflow explicitly communicates that there isn’t enough context to provide an answer, ensuring clarity and transparency in interactions.

---

### **Workflow Description**

1. **Input Validation Node**  
   - **Purpose**: Ensures the user has entered a valid question.  
   - **Flow**: If the input is valid, it proceeds to evaluate the query’s relevance; otherwise, it terminates with an error message.

2. **Context Provider Node**  
   - **Purpose**: Checks whether the question is specific to the guided project.  
     - For relevant questions, it provides predefined project-specific context.  
     - For unrelated questions, it sets the context to `null`.  
   - **Flow**: Always transitions to the question-answering step, whether or not context is available.

3. **LLM Question-Answering Node**  
   - **Purpose**: Uses the context (if available) to answer the question.  
     - If context is provided, it generates a detailed response.  
     - If context is `null`, it responds with: *"I don't have enough context to answer your question. Please ask about the guided project."*


In [4]:
# @title Auth State class
from typing import TypedDict, Optional

class QAState(TypedDict):
    question: Optional[str]
    context: Optional[str]
    answer: Optional[str]

In [5]:
qa_state_example = QAState(
    question= "what is the purpose of this guided project?",
    context= "This project focuses on building a chatbot using Python.",
    answer = None
)

In [6]:
for key, value in qa_state_example.items():
    print(f"{key}: {value}")

question: what is the purpose of this guided project?
context: This project focuses on building a chatbot using Python.
answer: None


#### **Defining the Input Validation Node**

<div style="text-align: center;">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/QNWh9LRDo4A3uF5cz4bXcQ/Screenshot%202024-12-20%20at%204-20-07%E2%80%AFPM-mh.png" alt="Screenshot" width="150">
</div>


In this node, we validate the user's input (the question). The node checks whether the question is provided and if it's not empty. If the question is empty, it returns an error message indicating that the question cannot be empty. If the question is valid, it proceeds to the next node.


In [7]:
def input_validation_node(state):
    # Extract the question from the state, and strip any leading or trailing spaces
    question = state.get("question", "").strip()

    # If the question is empty, return an error message indicating invalid input
    if not question:
        return {"valid": False, "error": "Question cannot be empty."}

    # If the question is valid, return valid status
    return {"valid": True}

In [8]:
input_validation_node(qa_state_example)

{'valid': True}

#### **Defining the Context Provider**

<div style="text-align: center;">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/htDP3RKH9b6X1NHSKME-YQ/Screenshot%202024-12-20%20at%204-20-07%E2%80%AFPM-mh%20-1-.png" alt="Screenshot" width="150">
</div>


This node checks if the question is related to the guided project. If it mentions "LangGraph" or "guided project," it provides the relevant context. Otherwise, it sets the context to `None':


In [9]:
def context_provider_node(state):
    question = state.get("question", "").lower()
    # Check if the question is related to the guided project
    if "langgraph" in question or "guided project" in question:
        context = (
            "This guided project is about using LangGraph, a Python library to design state-based workflows. "
            "LangGraph simplifies building complex applications by connecting modular nodes with conditional edges."
        )
        return {"context": context}
    # If unrelated, set context to null
    return {"context": None}

#### **Integrating LLM for QA Workflow**  

<div style="text-align: center;">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/Djs-AScfvwE8fnKButiPXg/Screenshot%202024-12-20%20at%204-20-07%E2%80%AFPM-mh%20-2-.png" alt="Screenshot" width="150">
</div>


In this step, we are building a node that utilizes an LLM (Large Language Model) to answer user questions based on the provided context. If the question is unrelated to the guided project, the node handles this gracefully by returning a predefined response. This approach uses OpenAI's gpt 3.5 turbo model through LangChain's `OpenAI` interface.


In [11]:
def llm_qa_node(state):
    # Extract the question and context from the state
    question = state.get("question", "")
    context = state.get("context", None)

    # Check for missing context and return a fallback response
    if not context:
        return {"answer": "I don't have enough context to answer your question."}

    # Construct the prompt dynamically
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer the question based on the provided context."

    # Use LangChain's ChatOpenAI to get the response
    try:
        response = llm.invoke(prompt)
        return {"answer": response.content.strip()}
    except Exception as e:
        return {"answer": f"An error occurred: {str(e)}"}

In [13]:
qa_workflow = StateGraph(QAState)

In [15]:
qa_workflow.add_node("InputNode", input_validation_node)
qa_workflow.add_node("ContextNode", context_provider_node)
qa_workflow.add_node("QANode", llm_qa_node)


<div style="text-align: center;">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/0K3mU7FSZFw0tnAXiN22ag/Screenshot%202024-12-20%20at%204-20-07%E2%80%AFPM-mh%20-3-.png" alt="Screenshot" width="150">
</div>


Now, we set the **entry point** for our QA workflow to the **`InputNode`**. This is the first node that will be executed when the workflow starts, ensuring that the user's input is validated before any other operations occur.


In [16]:
qa_workflow.set_entry_point("InputNode")

In [18]:
from langgraph.graph import END

qa_workflow.add_edge("InputNode", "ContextNode")
qa_workflow.add_edge("ContextNode", "QANode")
qa_workflow.add_edge("QANode", END)

In [19]:
qa_app = qa_workflow.compile()

In [20]:
qa_app.invoke({"question": "What is the weather today?"})

{'question': 'What is the weather today?',
 'context': None,
 'answer': "I don't have enough context to answer your question."}

In [21]:
qa_app.invoke({"question": "What is LangGraph?"})

{'question': 'What is LangGraph?',
 'context': 'This guided project is about using LangGraph, a Python library to design state-based workflows. LangGraph simplifies building complex applications by connecting modular nodes with conditional edges.',
 'answer': 'LangGraph is a Python library that simplifies building complex applications by connecting modular nodes with conditional edges to design state-based workflows.'}

In [22]:
qa_app.invoke({"question": "What is the best guided project?"})

{'question': 'What is the best guided project?',
 'context': 'This guided project is about using LangGraph, a Python library to design state-based workflows. LangGraph simplifies building complex applications by connecting modular nodes with conditional edges.',
 'answer': 'The best guided project in this context would be one that focuses on creating a robust state-based workflow using LangGraph. This project should involve designing a workflow with multiple modular nodes connected by conditional edges, demonstrating how LangGraph can simplify the process of building complex applications. By working through this guided project, users can gain a deeper understanding of how to effectively utilize LangGraph in their own projects.'}